In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
from __future__ import print_function

tf.set_random_seed(777)

In [2]:
sentence=("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set=list(set(sentence))
char_dic={w:i for i,w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size= len(char_set)
num_classes= len(char_set)
batch_size= 1
sequence_length= 10
learning_rate= 0.1

dataX=[]
dataY=[]


In [4]:
for i in range(0, len(sentence)-sequence_length):
    x_str=sentence[i:i+sequence_length]
    y_str=sentence[i+1:i+sequence_length+1]
    
    print(i,x_str,'->',y_str)
    x=[char_dic[c] for c in x_str]
    y=[char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
 
batch_size= len(dataX)
    

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [5]:

X=tf.placeholder(tf.int32,[None,sequence_length])
Y=tf.placeholder(tf.int32,[None,sequence_length])

x_one_hot = tf.one_hot(X,num_classes)
print(x_one_hot)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [6]:
def lstm_cell():
    cell=rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    return cell

In [7]:
multi_cells=rnn.MultiRNNCell([lstm_cell() for _  in range(2)],state_is_tuple=True )

In [8]:
outputs,_states=tf.nn.dynamic_rnn(multi_cells,x_one_hot, dtype=tf.float32)

In [10]:
x_for_fc =tf.reshape(outputs, [-1,hidden_size])
outputs=tf.contrib.layers.fully_connected(x_for_fc, num_classes,activation_fn=None)

In [11]:
outputs=tf.reshape(outputs,[batch_size,sequence_length, num_classes])
weights=tf.ones([batch_size, sequence_length])

In [12]:
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=Y,weights=weights)

In [13]:
mean_loss =tf.reduce_mean(sequence_loss)

In [14]:
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        _, l,results =sess.run([train,mean_loss, outputs], feed_dict={X:dataX, Y:dataY})
        for j, result in enumerate(results):
            index=np.argmax(result,axis=1)
            print(i,j,''.join([char_set[t] for t  in index]),l)


0 0 prphhhhhhh 3.21779
0 1 rahhhhhhhh 3.21779
0 2 ahhhhhyhhh 3.21779
0 3 yyyhhyhhhh 3.21779
0 4 ffffyhhhgg 3.21779
0 5 nnaaaggggg 3.21779
0 6 aaaaggggga 3.21779
0 7 drrmggggga 3.21779
0 8 whhhhhhhhh 3.21779
0 9 tggggaaaaa 3.21779
0 10 ogggaaaaaa 3.21779
0 11 aaaaaaaahh 3.21779
0 12 ooaaaaahhh 3.21779
0 13 fhhhhhhhhh 3.21779
0 14 aaaaakhhhh 3.21779
0 15 rrrrrpphhh 3.21779
0 16 nrrppphhhh 3.21779
0 17 ppppphhhhh 3.21779
0 18 ayyyhhhhhh 3.21779
0 19 yyyhhhhhhh 3.21779
0 20 ahhhhhhhhh 3.21779
0 21 whhhhhhhhh 3.21779
0 22 ahhhhhhhhh 3.21779
0 23 cccphhhhhh 3.21779
0 24 tppphhhhhh 3.21779
0 25 ppphhhhhhh 3.21779
0 26 phhhhhhhhh 3.21779
0 27 yhyyhhhhhh 3.21779
0 28 ahyhhhhhhy 3.21779
0 29 yythhhhhyy 3.21779
0 30 ffhhhhhhyg 3.21779
0 31 thhhhhyggg 3.21779
0 32 khhhyygggg 3.21779
0 33 oggggggggg 3.21779
0 34 ahyygggghh 3.21779
0 35 yyyygg phh 3.21779
0 36 ggggggghhh 3.21779
0 37 ntg   khhh 3.21779
0 38 tgggghhhhh 3.21779
0 39 aahhhhhhhh 3.21779
0 40 nknkhhhhhh 3.21779
0 41 phhhhhhhhh 3.21779
0 

3 58 tttttttttt 3.03566
3 59 tttttttttt 3.03566
3 60 tttttttttt 3.03566
3 61 tttttttttt 3.03566
3 62 tttttttttt 3.03566
3 63 tttttttttt 3.03566
3 64 tttttttttt 3.03566
3 65 tttttttttt 3.03566
3 66 tttttttttt 3.03566
3 67 tttttttttt 3.03566
3 68 tttttttttt 3.03566
3 69 tttttttttt 3.03566
3 70 tttttttttt 3.03566
3 71 tttttttttt 3.03566
3 72 tttttttttt 3.03566
3 73 tttttttttt 3.03566
3 74 tttttttttt 3.03566
3 75 tttttttttt 3.03566
3 76 tttttttttt 3.03566
3 77 tttttttttt 3.03566
3 78 tttttttttt 3.03566
3 79 tttttttttt 3.03566
3 80 tttttttttt 3.03566
3 81 tttttttttt 3.03566
3 82 tttttttttt 3.03566
3 83 tttttttttt 3.03566
3 84 tttttttttt 3.03566
3 85 tttttttttt 3.03566
3 86 tttttttttt 3.03566
3 87 tttttttttt 3.03566
3 88 tttttttttt 3.03566
3 89 tttttttttt 3.03566
3 90 tttttttttt 3.03566
3 91 tttttttttt 3.03566
3 92 tttttttttt 3.03566
3 93 tttttttttt 3.03566
3 94 tttttttttt 3.03566
3 95 tttttttttt 3.03566
3 96 tttttttttt 3.03566
3 97 tttttttttt 3.03566
3 98 tttttttttt 3.03566
3 99 tttttttttt 

6 128 eeeeeeeeee 2.93535
6 129 eeeeeeeeee 2.93535
6 130 eeeeeeeeee 2.93535
6 131 eee eeeeee 2.93535
6 132 ee eeeeeee 2.93535
6 133 e ee eeeee 2.93535
6 134 eee eeeeee 2.93535
6 135 eeeeeeeeee 2.93535
6 136    eeeeeee 2.93535
6 137 eeeeeeeee  2.93535
6 138 eeeeeeee e 2.93535
6 139   e eee ee 2.93535
6 140  e eee eee 2.93535
6 141 e eee eeee 2.93535
6 142 eeee eeeee 2.93535
6 143 eee eeeeee 2.93535
6 144    eeeeeee 2.93535
6 145   eeeeeeee 2.93535
6 146 eeeeeeeeee 2.93535
6 147 eeeeeeeeee 2.93535
6 148 eeeeeeeeee 2.93535
6 149 e eeeeeeee 2.93535
6 150 eeeeeeeeee 2.93535
6 151 eeeeeeeeee 2.93535
6 152  eeeeeeeee 2.93535
6 153 eeeeeeeeee 2.93535
6 154 eeee eeeee 2.93535
6 155 eee eeeeee 2.93535
6 156 e   eeeeee 2.93535
6 157 e  eeeeeee 2.93535
6 158 eeeeeeeeee 2.93535
6 159 eeeeeeeee  2.93535
6 160 eeeeeeee e 2.93535
6 161 eeeeeee ee 2.93535
6 162 eeeeee eee 2.93535
6 163   eee ee e 2.93535
6 164 e ee ee ee 2.93535
6 165 eee ee eee 2.93535
6 166 ee ee eeee 2.93535
6 167 e  e eeeee 2.93535


9 120            2.84855
9 121            2.84855
9 122            2.84855
9 123            2.84855
9 124            2.84855
9 125            2.84855
9 126            2.84855
9 127            2.84855
9 128            2.84855
9 129            2.84855
9 130            2.84855
9 131            2.84855
9 132            2.84855
9 133            2.84855
9 134            2.84855
9 135            2.84855
9 136            2.84855
9 137            2.84855
9 138            2.84855
9 139            2.84855
9 140            2.84855
9 141            2.84855
9 142            2.84855
9 143            2.84855
9 144            2.84855
9 145            2.84855
9 146            2.84855
9 147            2.84855
9 148            2.84855
9 149            2.84855
9 150            2.84855
9 151            2.84855
9 152            2.84855
9 153            2.84855
9 154            2.84855
9 155            2.84855
9 156            2.84855
9 157            2.84855
9 158            2.84855
9 159            2.84855


12 140            2.72746
12 141            2.72746
12 142            2.72746
12 143            2.72746
12 144            2.72746
12 145            2.72746
12 146            2.72746
12 147            2.72746
12 148            2.72746
12 149            2.72746
12 150            2.72746
12 151            2.72746
12 152            2.72746
12 153            2.72746
12 154            2.72746
12 155            2.72746
12 156            2.72746
12 157            2.72746
12 158            2.72746
12 159            2.72746
12 160            2.72746
12 161            2.72746
12 162            2.72746
12 163            2.72746
12 164            2.72746
12 165            2.72746
12 166            2.72746
12 167            2.72746
12 168            2.72746
12 169            2.72746
13 0            2.6676
13 1            2.6676
13 2            2.6676
13 3          o 2.6676
13 4         o  2.6676
13 5        o o 2.6676
13 6  o    o o  2.6676
13 7      o o   2.6676
13 8     o o    2.6676
13 9    o o  

...

170 80 n't assign 0.230788
170 81 dt assign  0.230788
170 82 t assign t 0.230788
170 83 hassign th 0.230788
170 84 tnsign the 0.230788
170 85 nsign them 0.230788
170 86   gn them  0.230788
170 87  tn them t 0.230788
170 88 mn them ta 0.230788
170 89   them tas 0.230788
170 90 dthem task 0.230788
170 91 toem tasks 0.230788
170 92 hem tasks  0.230788
170 93 em tosks a 0.230788
170 94   tasks an 0.230788
170 95  tosks and 0.230788
170 96 tosks and  0.230788
170 97 hsks and w 0.230788
170 98 nss and wo 0.230788
170 99  s and wor 0.230788
170 100 , and work 0.230788
170 101  and work, 0.230788
170 102 tnd dork,  0.230788
170 103 nd dork, b 0.230788
170 104 d work, bu 0.230788
170 105  aork, but 0.230788
170 106 took, but  0.230788
170 107 ook, but r 0.230788
170 108 nk, but ra 0.230788
170 109  , but rat 0.230788
170 110 , but rath 0.230788
170 111  but rathe 0.230788
170 112 tui rather 0.230788
170 113 ui rather  0.230788
170 114   rather t 0.230788
170 115 haather te 0.230788
170 116 tath

173 115 haather te 0.230877
173 116 tather tea 0.230877
173 117  ther teac 0.230877
173 118 nher teach 0.230877
173 119 her toach  0.230877
173 120 em toach t 0.230877
173 121   toach th 0.230877
173 122  toach the 0.230877
173 123 toach them 0.230877
173 124 hach them  0.230877
173 125  ch them t 0.230877
173 126 nh them to 0.230877
173 127 o them to  0.230877
173 128 ethem to l 0.230877
173 129 toem to lo 0.230877
173 130 her to lon 0.230877
173 131 em to long 0.230877
173 132   to long  0.230877
173 133  to long f 0.230877
173 134 to cong fo 0.230877
173 135 h cong for 0.230877
173 136  cong for  0.230877
173 137 tong for t 0.230877
173 138 eng for th 0.230877
173 139  ' for the 0.230877
173 140 d for the  0.230877
173 141  for the e 0.230877
173 142 tor the en 0.230877
173 143 or the end 0.230877
173 144   the endl 0.230877
173 145  toe endle 0.230877
173 146 toemendles 0.230877
173 147 herendless 0.230877
173 148 emendless  0.230877
173 149  endless i 0.230877
173 150 tndless im 0

176 121 n toach th 0.230691
176 122  thach the 0.230691
176 123 toach them 0.230691
176 124 hach them  0.230691
176 125 nch them t 0.230691
176 126 nh them to 0.230691
176 127 o them to  0.230691
176 128 ethem to l 0.230691
176 129 toem ta lo 0.230691
176 130 hem ta lon 0.230691
176 131 em ta long 0.230691
176 132 n ta long  0.230691
176 133  ta long f 0.230691
176 134 to bong fo 0.230691
176 135 h bong for 0.230691
176 136  cong for  0.230691
176 137 tong for t 0.230691
176 138 eng for th 0.230691
176 139  ' for the 0.230691
176 140 d for the  0.230691
176 141 nfor the e 0.230691
176 142 tor the en 0.230691
176 143  r the end 0.230691
176 144  kthe endl 0.230691
176 145  the endle 0.230691
176 146 toemendles 0.230691
176 147 hemendless 0.230691
176 148 emendless  0.230691
176 149 ntndless i 0.230691
176 150 tndless im 0.230691
176 151 nsless imm 0.230691
176 152 d ess imme 0.230691
176 153  ess immen 0.230691
176 154 ess immens 0.230691
176 155 ns immensi 0.230691
176 156   immensit 0

179 119 hem toach  0.230399
179 120 em toach t 0.230399
179 121 n toach th 0.230399
179 122  thach the 0.230399
179 123 toach them 0.230399
179 124 hach them  0.230399
179 125 nch them t 0.230399
179 126 nh them to 0.230399
179 127 o them to  0.230399
179 128 ethem to l 0.230399
179 129 toem ta lo 0.230399
179 130 hem ta lon 0.230399
179 131 em ta long 0.230399
179 132 n ta long  0.230399
179 133  ta long f 0.230399
179 134 to cong fo 0.230399
179 135 h bong for 0.230399
179 136  bong for  0.230399
179 137 tong for t 0.230399
179 138 eng for th 0.230399
179 139  ' for the 0.230399
179 140 d for the  0.230399
179 141 nfor the e 0.230399
179 142 tor the en 0.230399
179 143  r the end 0.230399
179 144  kthe endl 0.230399
179 145  the endle 0.230399
179 146 toemendles 0.230399
179 147 hemendless 0.230399
179 148 emendless  0.230399
179 149 ntndless i 0.230399
179 150 tndless im 0.230399
179 151 nsless imm 0.230399
179 152 d ess imme 0.230399
179 153  ess immen 0.230399
179 154 ess immens 0

181 125 rch them t 0.230458
181 126 nh them to 0.230458
181 127 o them to  0.230458
181 128 ethem to l 0.230458
181 129 toem ta lo 0.230458
181 130 hem ta lon 0.230458
181 131 em ta long 0.230458
181 132 r ta long  0.230458
181 133  ta long f 0.230458
181 134 to cong fo 0.230458
181 135 h bong for 0.230458
181 136 nbong for  0.230458
181 137 tong for t 0.230458
181 138 eng for th 0.230458
181 139 n' for the 0.230458
181 140 d for the  0.230458
181 141 efor the e 0.230458
181 142 tor the en 0.230458
181 143  r the end 0.230458
181 144 nkthe endl 0.230458
181 145  tee endle 0.230458
181 146 toemendles 0.230458
181 147 hemendless 0.230458
181 148 emendless  0.230458
181 149 rendless i 0.230458
181 150 tndless im 0.230458
181 151 rsless imm 0.230458
181 152 d ess imme 0.230458
181 153  ess immen 0.230458
181 154 ecs immens 0.230458
181 155 rs immensi 0.230458
181 156   immensit 0.230458
181 157  ammensity 0.230458
181 158 tmmensity  0.230458
181 159 mmensity o 0.230458
181 160  ensity of 0

In [ ]:
results=sess.run(outputs,feed_dict={X:dataX})


In [ ]:
for j,result in enumerate(results):
    index= np.argmax(result,axis=1)
    if j is  0:
        print(''.join([char_set[t] for t in index] ), end='')
    else :
        print(char_set[index[-1]], end='')
        